In [1]:
#======================================================================================================
#                                              Packages
#======================================================================================================

# Import packages

import pandas as pd
import numpy as np

# importing os module
import os
import pymysql
import time

from pandas import ExcelWriter


In [2]:
#======================================================================================================
#                                              Functions
#======================================================================================================

def get_sheets_xls_file(path, file_name):
    """Extract all the sheets of xls file
        returns a list of sheets
    
    Example of path and file_name
    path = os.path.dirname(os.getcwd()) + "\\data\\" 
    file_name = "BT_Cgr_Es_2022-01_Source.xlsx" 
    """
    all_sheets = pd.read_excel(path+file_name, sheet_name= None)
    
    return list(all_sheets.keys())



def get_one_xls_file_str(path, file_name, sheet_name): #header=0, all files have the same format and start at the same row
    """ Get dataframe from xls file, for a particular sheet_name, header=0, we can set it as an argument
        returns dataframe
    """
    df = pd.read_excel(path+file_name, sheet_name= sheet_name, 
                  skiprows=None, header=0, nrows=1)
    
    col_str_dic = {column:str for column in list(df)}
    #print("col_str_dic  ", col_str_dic)
    
    dfN = pd.read_excel(path+file_name, sheet_name= sheet_name, 
                  skiprows=None, header=0, dtype=col_str_dic) 
    
    return dfN


In [3]:
#======================================================================================================
#                                      Set Pathfiles directories
#======================================================================================================

# Source path E-catalogue
path_cat =  r"\\NAS-WORK\E - Catalogue\Lavigna\ECAT 09-2022"

#path_source = 'C:\\Users\\romai\\Desktop\\Kenza\\Base Données\\MATRICE\\Plomberie\\'  

path_controle = r"\\NAS-WORK\Base Données\MATRICE\MATRICE_LAVIGNA"

## 1. Connecting to BDD with pymysql

In [4]:

#Connexion VPN
PASSWORD =  "J74mTu53#"
HOSTNAME = '10.4.0.200'
USERNAME = 'iabtp'
PORT = 3306

    
#database connection
connection = pymysql.connect(host = HOSTNAME, 
                             user= USERNAME, 
                             password= PASSWORD, 
                             port = PORT
                              )



In [5]:
DATABASE_1 = 'MATRICE_CVC'
table_name_1 = 'Correspondances'

## GET MATRICE_CVC

In [6]:
start_time = time.time()

connection = pymysql.connect(host = HOSTNAME, 
                             user= USERNAME, 
                             password= PASSWORD, 
                             port = PORT
                              )

sql = f'''SELECT tb_1.* 
                 FROM {DATABASE_1}.{table_name_1} tb_1
      '''

cursor = connection.cursor()

run_query = cursor.execute(sql)

num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]

#print("field_names  ", field_names)
#print(" num_fields  ", num_fields)

L = cursor.fetchall()
result = pd.DataFrame(L)
result.columns = field_names

print("result.shape :  ", result.shape)

print("--- %s seconds ---" % (time.time() - start_time))

result.head(3)

result.shape :   (47159, 14)
--- 2.4091227054595947 seconds ---


,Famille,SF,SF2,Ref_IABTP,Distributeur,Ref_Distr,Fabricant,Ref_Fabricant,Designation,Dimension,Unite,Date,Remarques Ref_LAVIGNA,IsValid
0,Acier Carbone,None,None,AC1000,Aredis,EZ15,None,None,Tube Acier Carbonne Lg 6ML,15,ML,2022-01-01,IATGEZ0007,1
1,Acier Carbone,None,None,AC1000,First,TAC015,None,None,Tube Acier Carbonne Lg 6ML,15,ML,2022-01-01,IATGEZ0007,1
2,Acier Carbone,None,None,AC1000,Cgr,RSEZ2431515,None,None,Tube Acier Carbonne Lg 6ML,15,ML,2022-01-01,IATGEZ0007,1


In [7]:
# Make a copy of dataframe
df_MAT = result.copy()

In [8]:
df_MAT.shape

(47159, 14)

In [9]:
def CheckElements_Distributeur(df_matrix = pd.DataFrame(), new_df = pd.DataFrame(), famille = str):
   
    L= []
    
    # We need to automatize the process, even for the name of columns "Prix"
    #hence the following syntax
    RefsList_0 = [i for i in list(new_df.columns)\
                  if i.startswith('Ref_') and not i.startswith('Ref_IABTP')]

    RefsList = [i.replace('Ref_', '') for i in RefsList_0]

    #print("Colonnes 'Distributeurs' :", RefsList)


    myList1 = list(df_matrix[df_matrix['Famille'] == famille]['Distributeur'].unique())
    
   

    for i in RefsList:
        L.append(i in myList1)
        
    K_new = []
    K_MAT = []
    for i,j in enumerate(RefsList_0):
        K_new.append(new_df.loc[1:, j].count())
        
        if (famille == "Multicouche") and (RefsList[i] == "Pum"): 
            
            df_temp = df_matrix[(df_matrix['Famille'] == famille) 
                                & (df_matrix['Distributeur'] == RefsList[i])
                                & (df_matrix['Fabricant'] == "Nicoll")]
    
        else:
    
            df_temp = df_matrix[(df_matrix['Famille'] == famille) & (df_matrix['Distributeur'] == RefsList[i]) ]
        
       # iabtp_MAT = list(df_temp['Ref_IABTP'])
       # refDist = list(df_temp['Ref_Distr'])
                      
        K_MAT.append(df_temp['Ref_Distr'].count())

    return np.all(L), RefsList, K_new, K_MAT #, iabtp, refDist

#check, RefsList, Count_New, Count_MAT =  CheckElements_Distributeur(df_matrix = data, new_df = lavigna, famille = 'Acier Carbone')

In [10]:
def merge_data_on_col(df1 = pd.DataFrame(), df2= pd.DataFrame(), col = "Ref_IABTP",
                      dataname1 = '_LAV', dataname2 = '_MAT'):
    
    
   # print(col)
    if len(col) > 1 :
        COL1 = col[1]
    elif len([col]) ==1 :
        COL1 = col
           
    res = df1[~(df1[COL1].isnull()) ].merge(df2[~(df2[COL1].isnull()) ],
              on = col,
             indicator=True,
                           how = "outer",
              suffixes=(dataname1, dataname2))

        
    b = res[res["_merge"] == 'both']
    bc = b[COL1].count()


    l = res[res["_merge"] == 'left_only'] # CAT
    lc = l[COL1].count()

    r = res[res["_merge"] == 'right_only'] # Corresp
    rc = r[COL1].count()

    return res, bc, lc, rc   #, b, l, r

In [11]:
def count_data_on_col(df= pd.DataFrame(), col = 'Ref_IABTP'):
    
        
    b = df[df["_merge"] == 'both']
    bc = b[col].count()


    l = df[df["_merge"] == 'left_only'] # CAT
    lc = l[col].count()

    r = df[df["_merge"] == 'right_only'] # Corresp
    rc = r[col].dropna().count()

    return bc, lc, rc

In [12]:
def GetMultiIndexCases(ind1 ,
                       ind2 ,
                       source = pd.DataFrame(),
                       matrix = pd.DataFrame() ):



    case1 = source.loc[ ind1.isin(ind2)]
    case2 = source.loc[~ ind1.isin(ind2)]


    case3 = matrix.loc[ind2.isin(ind1)]
    case4 = matrix.loc[~ ind2.isin(ind1)]


    return case1, case2, case3, case4 
 

# Get data "E-CAT Lavigna"

In [13]:
#dataframe = pd.read_excel(f"{path_MAT}Matrice_V4.xlsx")
dataframe_sheets = get_sheets_xls_file(path = path_cat, file_name = "\E-CAT_Lavigna_VMC-Plomberie_2022-09-19.xlsx")
    
sh = dataframe_sheets[0]
dataframe = get_one_xls_file_str(path = path_cat, file_name = "\E-CAT_Lavigna_VMC-Plomberie_2022-09-19.xlsx",
                                  sheet_name= sh
                  )

dataframe.shape

(9759, 20)

In [14]:
# Make a copy of dataframe
res = dataframe.copy()

In [15]:
res.shape

(9759, 20)

In [16]:
# Only get Famille Plomberie
df_CAT_0 = res[res.Famille == "Plomberie"]
df_CAT_0.shape

(7931, 20)

In [17]:
# Read the 2 first rows of the dataframe
df_CAT_0.head(2)

,Nom du Client,Réf. IA BTP,Fournisseur,Réf. Fournisseur,Fabricant,Réf. Fabricant,Famille,Sous-Famille,Désignation,PU_HT,Unite Vente,Eco Taxe,Conditionnement,Délai,Url,Date Validité,Dérogation,Date de Validité Dérogation,Pdf Link,Spécificités techniques
378,Lavigna,AC1015,Andrety,50355,N.C.,N.C.,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d18,3.28,U,N.C.,N.C.,/ jour,N.C.,2022-12-31 00:00:00,N.C.,N.C.,N.C.,N.C.
379,Lavigna,AC1016,Andrety,50356,N.C.,N.C.,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d22,3.53,U,N.C.,N.C.,/ jour,N.C.,2022-12-31 00:00:00,N.C.,N.C.,N.C.,N.C.


In [18]:
df_CAT_0.loc[:, "Fam"] = df_CAT_0["Sous-Famille"].apply(lambda s:s.split(";")[0])
df_CAT_0.loc[:, "Fam"].sort_values().unique()

C:\Users\romai\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


array(['Acier Carbone', 'Acier Inox', 'Base tarifaire Aquatherm',
       'Calo Normal', 'Calo Prefendu', 'Chevillerie Boulonnerie',
       'Collier', 'Collier Calo', 'Cuivre A Sertir Eau',
       'Cuivre A Sertir Gaz', 'Multicouche', 'Per A Sertir',
       'Per Raccord Clarinette', 'Pvc Eu', 'Raccord Cuivre',
       'Raccord Laiton', 'Robinetterie Batiment', 'Supportage'],
      dtype=object)

In [19]:
list_ = df_CAT_0["Fam"].sort_values().unique()
len(list_)

18

In [20]:
# Read the 2 first rows of the dataframe
df_CAT_0.head(2)

,Nom du Client,Réf. IA BTP,Fournisseur,Réf. Fournisseur,Fabricant,Réf. Fabricant,Famille,Sous-Famille,Désignation,PU_HT,...,Eco Taxe,Conditionnement,Délai,Url,Date Validité,Dérogation,Date de Validité Dérogation,Pdf Link,Spécificités techniques,Fam
378,Lavigna,AC1015,Andrety,50355,N.C.,N.C.,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d18,3.28,...,N.C.,N.C.,/ jour,N.C.,2022-12-31 00:00:00,N.C.,N.C.,N.C.,N.C.,Acier Carbone
379,Lavigna,AC1016,Andrety,50356,N.C.,N.C.,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d22,3.53,...,N.C.,N.C.,/ jour,N.C.,2022-12-31 00:00:00,N.C.,N.C.,N.C.,N.C.,Acier Carbone


In [21]:
df_CAT_0[(df_CAT_0.Fam == "Base tarifaire Aquatherm")].shape

(1166, 21)

In [22]:
# Only get Famille Plomberie
df_CAT = df_CAT_0[~(df_CAT_0.Fam == "Base tarifaire Aquatherm")]
df_CAT.shape

(6765, 21)

In [23]:
list_ = df_CAT["Fam"].sort_values().unique()
len(list_)

17

In [24]:
# Get list "Familles"
listFamilles = list(list_)
listFamilles[:2]

['Acier Carbone', 'Acier Inox']

In [25]:
df_CAT.head(1)

,Nom du Client,Réf. IA BTP,Fournisseur,Réf. Fournisseur,Fabricant,Réf. Fabricant,Famille,Sous-Famille,Désignation,PU_HT,...,Eco Taxe,Conditionnement,Délai,Url,Date Validité,Dérogation,Date de Validité Dérogation,Pdf Link,Spécificités techniques,Fam
378,Lavigna,AC1015,Andrety,50355,N.C.,N.C.,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d18,3.28,...,N.C.,N.C.,/ jour,N.C.,2022-12-31 00:00:00,N.C.,N.C.,N.C.,N.C.,Acier Carbone


In [32]:
df_CAT.columns

Index(['Nom du Client', 'Réf. IA BTP', 'Fournisseur', 'Réf. Fournisseur',
       'Fabricant', 'Réf. Fabricant', 'Famille', 'Sous-Famille', 'Désignation',
       'PU_HT', 'Unite Vente', 'Eco Taxe', 'Conditionnement', 'Délai', 'Url',
       'Date Validité', 'Dérogation', 'Date de Validité Dérogation',
       'Pdf Link', 'Spécificités techniques', 'Fam'],
      dtype='object')

In [ ]:

p1 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. Fournisseur'].count()]).T

p1.columns = ["Ref_Fo (E-CAT)"]


q1 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. IA BTP'].count()]).T

q1.columns = ["Ref_IABTP (E-CAT)"]

q2 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. IA BTP'].count()]).T

q2.columns = ["Ref_IABTP (E-CAT)"]

In [26]:
#H = df_CAT.groupby(["Fam"]).agg({"Fournisseur" : list}).reset_index()

In [27]:
H = df_CAT.groupby(["Fam"])["Fournisseur"].agg(lambda x:x.unique() ).reset_index()
H.rename(columns = {"Fam" : "Famille", 
                   "Fournisseur" : "Distributeur"}, inplace =True)

In [28]:
H1 = H.explode(column = ["Distributeur"])
H1.head(2)

,Famille,Distributeur
0,Acier Carbone,Andrety
0,Acier Carbone,Aredis


In [29]:
df_MAT.Famille.unique()

array(['Acier Carbone', 'Acier Inox', 'Bati Support', 'Calo Normal',
       'Calo Prefendu', 'Chevillerie Boulonnerie', 'Collier',
       'Collier Calo', 'Conso Clim', 'Consommable',
       'Filtre à Boue Magnetique', 'Filtre Cintropur', 'Flexible Inox',
       'Flexible Sanitaire Chauffage', 'Gaz', 'Laine De Verre',
       'Mepla Geberit Multicouche', 'Multicouche', 'PER à Glissement',
       'PVC C', 'PVC EU', 'PVC Pression', 'Raccord Acier',
       'Raccord Cuivre', 'Raccord Laiton', 'Robinetterie Batiment',
       'Supportage', 'Tube Acier', 'Vase Expansion Chauffage',
       'Cuivre à Sertir Gaz', 'Cuivre à Sertir Eau', 'PER à Sertir',
       'Electro Portatif'], dtype=object)

In [30]:
df_CAT.Fam.unique()

array(['Acier Carbone', 'Multicouche', 'Per Raccord Clarinette', 'Pvc Eu',
       'Raccord Cuivre', 'Raccord Laiton', 'Robinetterie Batiment',
       'Acier Inox', 'Calo Normal', 'Calo Prefendu', 'Per A Sertir',
       'Supportage', 'Cuivre A Sertir Eau', 'Cuivre A Sertir Gaz',
       'Collier', 'Chevillerie Boulonnerie', 'Collier Calo'], dtype=object)

In [31]:
dict_Distr = {"PER à Glissement" : "Per Raccord Clarinette",
                         "Cuivre à Sertir Eau" : "Cuivre A Sertir Eau" ,
               "Cuivre à Sertir Gaz" : "Cuivre A Sertir Gaz" ,
                         "PER à Sertir" : "Per A Sertir",
                         "PVC EU" : "Pvc Eu"      
                        }

df_MAT.Famille.replace(dict_Distr, inplace = True)
df_MAT.Famille.unique()

array(['Acier Carbone', 'Acier Inox', 'Bati Support', 'Calo Normal',
       'Calo Prefendu', 'Chevillerie Boulonnerie', 'Collier',
       'Collier Calo', 'Conso Clim', 'Consommable',
       'Filtre à Boue Magnetique', 'Filtre Cintropur', 'Flexible Inox',
       'Flexible Sanitaire Chauffage', 'Gaz', 'Laine De Verre',
       'Mepla Geberit Multicouche', 'Multicouche',
       'Per Raccord Clarinette', 'PVC C', 'Pvc Eu', 'PVC Pression',
       'Raccord Acier', 'Raccord Cuivre', 'Raccord Laiton',
       'Robinetterie Batiment', 'Supportage', 'Tube Acier',
       'Vase Expansion Chauffage', 'Cuivre A Sertir Gaz',
       'Cuivre A Sertir Eau', 'Per A Sertir', 'Electro Portatif'],
      dtype=object)

In [520]:
#  Filter required for E-CAT

keys = list(H1.columns.values)
print("keys  : ", keys)

i1 = df_MAT.set_index(keys).index
i2 = H1.set_index(keys).index

# Below, the syntax we will use to save the E-CAT
# Reminder, as explained at the beginning, we need to filter data to obtain the final version of E-CAT 

print("MATRICE_CVC.shape : ", df_MAT.shape ) 

print("MATRICE_CVC Filtré.shape : ", df_MAT[i1.isin(i2)].shape ) 

keys  :  ['Famille', 'Distributeur']
MATRICE_CVC.shape :  (47159, 14)
MATRICE_CVC Filtré.shape :  (11878, 14)


In [521]:
mat_lav_0 = df_MAT[i1.isin(i2)]
mat_lav_0.head(2)

,Famille,SF,SF2,Ref_IABTP,Distributeur,Ref_Distr,Fabricant,Ref_Fabricant,Designation,Dimension,Unite,Date,Remarques Ref_LAVIGNA,IsValid
0,Acier Carbone,None,None,AC1000,Aredis,EZ15,None,None,Tube Acier Carbonne Lg 6ML,15,ML,2022-01-01,IATGEZ0007,1
1,Acier Carbone,None,None,AC1000,First,TAC015,None,None,Tube Acier Carbonne Lg 6ML,15,ML,2022-01-01,IATGEZ0007,1


In [522]:
mat_lav_0.Fabricant.unique()

array([None, 'Sagi Kflex', 'Armacell', 'Hitfix', 'Scellit', 'Spit',
       'Arma Decoup', 'Nicoll', 'Valsir', 'Wavin', 'Geb', 'Gurtner',
       'Sferaco', 'Itron France', 'Riquier', 'Ibp Groupe',
       'Watts Industries', 'Comap France S.a.', 'Giacomini', 'Rbm France',
       'Ste Flamco Flexcon', 'Oventrop', 'Atusa France Eurl', 'Thermador',
       'AIPS', 'Viega', 'Dynaflu'], dtype=object)

In [608]:
MAT_LAVIGNA = mat_lav_0[ ~ ((mat_lav_0.Famille == "Multicouche") 
                                 & (mat_lav_0.Distributeur == "Pum")
                                 & ((mat_lav_0.Fabricant == "Valsir")  | (mat_lav_0.Fabricant).isnull()) )
                             ]
print(" MATRICE LAVIGNA SHAPE-->", MAT_LAVIGNA.shape)

 MATRICE LAVIGNA SHAPE--> (11697, 14)


# MATRICE Personnalisée (ES)

In [609]:
path_controle

'\\\\NAS-WORK\\Base Données\\MATRICE\\MATRICE_LAVIGNA'

In [610]:
client = "Lavigna"

In [611]:
# save MATRICE
MAT_LAVIGNA.to_excel(f"{path_controle}\MATRICE_{client}_3.xlsx", 
               sheet_name="MATRICE", 
           index=False) 

In [612]:
MAT_LAVIGNA.shape

(11697, 14)

In [613]:
df_MAT['Ref_IABTP'].count(), MAT_LAVIGNA['Ref_IABTP'].count(), MAT_LAVIGNA['Ref_Distr'].count()

(47159, 11697, 7863)

In [614]:
L = []
L_1 = []
L_2 = []

for i in list(MAT_LAVIGNA.Famille.unique())[:]:
    
    
    dt_perso_0 = MAT_LAVIGNA[ (MAT_LAVIGNA.Famille == i) 
                             ]
    
    dt_whole_0 = df_MAT[ (df_MAT.Famille == i) 
                             ]
    
    for j in list(dt_perso_0.Distributeur.unique()):            
                      
        dt_perso_1 = dt_perso_0[ (dt_perso_0.Distributeur == j)  
                               ]
        
        dt_whole_1 = dt_whole_0[ (dt_whole_0.Distributeur == j)  
                               ]
        
        if i == "Multicouche" and j == "Pum" :
            
            
            dt_perso = dt_perso_1[ (dt_perso_1.Distributeur == j)  
                                  & (dt_perso_1.Fabricant == "Nicoll")
                               ]
        
            dt_whole = dt_whole_1[ (dt_whole_1.Distributeur == j)  
                                 & (dt_whole_1.Fabricant == "Nicoll")
                                 ]
                         

            res_1, bc_1, lc_1, rc_1= merge_data_on_col(df1 = dt_perso,
                                                      df2= dt_whole,
                                                      col = ['Ref_IABTP', 'Ref_Distr'], #"Ref_Distr",
                                                      dataname1 = '_LAV', 
                                                      dataname2 = '_MAT') 
        

            BB = res_1[res_1["_merge"] == 'both']
            BC_1 = BB['Ref_IABTP'].count()
            BC_2 = BB['Ref_Distr'].count()
            
            BC = min(BC_1, BC_2)

            LL = res_1[~(res_1["_merge"] == 'both')]
            LC_1 = LL['Ref_IABTP'].count()
            LC_2 = LL['Ref_Distr'].count()
            
            
            L_1.append([i, j, dt_whole["Ref_IABTP"].count(), dt_perso["Ref_IABTP"].count(),
                        dt_whole["Ref_Distr"].count(), dt_perso["Ref_Distr"].count(),
                        BC, LC]) #, RC
        
        
        else:
            
            dt_perso = dt_perso_1

            dt_whole = dt_whole_1

                #print(i, j, dt_perso_0.shape, dt_whole_0.shape, dt_perso.shape, dt_whole.shape)

            #display(dt_perso.head(2))
            #print(dt_perso.shape)

            #print([i, j, dt_perso["Ref_IABTP"].count(), dt_whole["Ref_IABTP"].count(), bc, lc, rc]) 

            res_1, bc_1, lc_1, rc_1= merge_data_on_col(df1 = dt_perso,
                                                      df2= dt_whole,
                                                      col = ['Ref_IABTP', 'Ref_Distr'], #"Ref_Distr",
                                                      dataname1 = '_LAV', 
                                                      dataname2 = '_MAT') 
        

            BB = res_1[res_1["_merge"] == 'both']
            BC_1 = BB['Ref_IABTP'].count()
            BC_2 = BB['Ref_Distr'].count()
            
            BC = min(BC_1, BC_2)

            LL = res_1[~(res_1["_merge"] == 'both')]
            LC_1 = LL['Ref_IABTP'].count()
            LC_2 = LL['Ref_Distr'].count()
            
            LC = min(LC_1, LC_2)

            #RR = res_1[res_1["_merge"] == 'right_only']
            #RC = RR['Ref_Distr'].count()

            #print(i,j,BC_1, BC_2,LC_1, LC_2)

            L_1.append([i, j, dt_whole["Ref_IABTP"].count(), dt_perso["Ref_IABTP"].count(),
                        dt_whole["Ref_Distr"].count(), dt_perso["Ref_Distr"].count(),
                        BC, LC]) #, RC


In [615]:
Controle_1 = pd.DataFrame(L_1, columns = ['Famille', 
                           'Distributeur',
                          'Nbre Ref_IABTP (MATRICE CVC)',
                          'Nbre Ref_IABTP (MATRICE Lavigna)',
                           'Nbre Ref_Distr (MATRICE CVC)',
                            'Nbre Ref_Distr (MATRICE Lavigna)',
                            'Nbre de (Ref_IABTP, Ref_Distr) MATCHÉES ',
                           'Nbre de (Ref_IABTP, Ref_Distr) Non MATCHÉES'
                          ])
Controle_1

,Famille,Distributeur,Nbre Ref_IABTP (MATRICE CVC),Nbre Ref_IABTP (MATRICE Lavigna),Nbre Ref_Distr (MATRICE CVC),Nbre Ref_Distr (MATRICE Lavigna),"Nbre de (Ref_IABTP, Ref_Distr) MATCHÉES","Nbre de (Ref_IABTP, Ref_Distr) Non MATCHÉES"
0,Acier Carbone,Aredis,222,222,222,222,222,0
1,Acier Carbone,First,222,222,221,221,221,0
2,Acier Carbone,Cgr,222,222,191,191,191,0
3,Acier Carbone,Sgf,222,222,191,191,191,0
4,Acier Carbone,Andrety,222,222,117,117,117,0
...,...,...,...,...,...,...,...,...
59,Cuivre A Sertir Eau,Sgf,776,776,776,776,1552,0
60,Cuivre A Sertir Eau,Cgr,388,388,64,64,64,0
61,Per A Sertir,Sgf,64,64,64,64,64,0
62,Per A Sertir,Cgr,64,64,50,50,50,0


In [616]:
filepath = f"{path_controle}/Etat_Des_Lieux_MAT-{client}_3.xlsx"
    
with pd.ExcelWriter(filepath) as writer:
    
    
    Controle_1.to_excel(writer, 
               sheet_name="Croisement(Ref_IABTP,Ref_Distr)", 
           index=False) 
    
    
    

# Contrôle MATRICE / MATRICE Personnalisée Client

In [617]:
mask_cols = ['Famille', 'Ref_IABTP', 
              'Distributeur' , 'Ref_Distr', 
            'Fabricant', 'Ref_Fabricant' ]

In [618]:
df1.shape

(11788, 6)

In [619]:
df1 = MAT_LAVIGNA[mask_cols]
df2 = df_MAT[mask_cols]

dataname1 = '_LAV'
dataname2 = '_MAT'

COL1 = ['Famille', 'Ref_IABTP', 'Distributeur' , 'Ref_Distr']
    
F = df1[~(df1[mask_cols].isnull()) ].merge(df2[~(df2[mask_cols].isnull()) ],
              on = COL1,
             indicator=True,
                           how = "left",
              suffixes=(dataname1, dataname2))
F

,Famille,Ref_IABTP,Distributeur,Ref_Distr,Fabricant_LAV,Ref_Fabricant_LAV,Fabricant_MAT,Ref_Fabricant_MAT,_merge
0,Acier Carbone,AC1000,Aredis,EZ15,NaN,NaN,NaN,NaN,both
1,Acier Carbone,AC1000,First,TAC015,NaN,NaN,NaN,NaN,both
2,Acier Carbone,AC1000,Cgr,RSEZ2431515,NaN,NaN,NaN,NaN,both
3,Acier Carbone,AC1000,Sgf,P640008,NaN,NaN,NaN,NaN,both
4,Acier Carbone,AC1000,Andrety,49410,NaN,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...
12488,Per A Sertir,PR1200,Cgr,CRSPTR201620,NaN,NaN,NaN,NaN,both
12489,Per A Sertir,PR1200,Aredis,39111201620,NaN,NaN,NaN,NaN,both
12490,Per A Sertir,PR1201,Sgf,P624347,Riquier,18179,Riquier,18179,both
12491,Per A Sertir,PR1201,Cgr,CRSPTR202016,NaN,NaN,NaN,NaN,both


In [620]:
F.shape

(12493, 9)

In [621]:
G = df1[~(df1[mask_cols].isnull()) ].merge(df2[~(df2[mask_cols].isnull()) ],
              on = mask_cols,
             indicator=True,
                           how = "left",
              suffixes=(dataname1, dataname2))
G

,Famille,Ref_IABTP,Distributeur,Ref_Distr,Fabricant,Ref_Fabricant,_merge
0,Acier Carbone,AC1000,Aredis,EZ15,NaN,NaN,both
1,Acier Carbone,AC1000,First,TAC015,NaN,NaN,both
2,Acier Carbone,AC1000,Cgr,RSEZ2431515,NaN,NaN,both
3,Acier Carbone,AC1000,Sgf,P640008,NaN,NaN,both
4,Acier Carbone,AC1000,Andrety,49410,NaN,NaN,both
...,...,...,...,...,...,...,...
11692,Per A Sertir,PR1200,Cgr,CRSPTR201620,NaN,NaN,both
11693,Per A Sertir,PR1200,Aredis,39111201620,NaN,NaN,both
11694,Per A Sertir,PR1201,Sgf,P624347,Riquier,18179,both
11695,Per A Sertir,PR1201,Cgr,CRSPTR202016,NaN,NaN,both


In [622]:
# save MATRICE
G.to_excel(fr"{path_controle}\Etat_Lieux_test_{client}__3.xlsx", 
               sheet_name="MATRICE", 
           index=False) 

In [623]:
G["_merge"].unique()

['both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [624]:
b = G[G["_merge"] == 'both']
b.count()

Famille          11697
Ref_IABTP        11697
Distributeur     11697
Ref_Distr         7863
Fabricant         2393
Ref_Fabricant      917
_merge           11697
dtype: int64

In [625]:
df1.count()

Famille          11697
Ref_IABTP        11697
Distributeur     11697
Ref_Distr         7863
Fabricant         2393
Ref_Fabricant      917
dtype: int64

In [626]:
df2.count()

Famille          47159
Ref_IABTP        47159
Distributeur     47153
Ref_Distr        16737
Fabricant         3609
Ref_Fabricant     1775
dtype: int64

In [627]:
p1 = pd.DataFrame([b.groupby(['Famille', 'Distributeur'], dropna=False)['Ref_Distr'].size()]).T
p1

Ref_Distr
Famille               Distributeur            
Acier Carbone         Andrety              222
                      Aredis               222
                      Cgr                  222
                      First                222
                      Sgf                  222
...                                        ...
Robinetterie Batiment Sgf                  186
Supportage            Aredis               112
                      First                112
                      Mps-Prolians         112
                      Outillage-Mer        112

[64 rows x 1 columns]

In [628]:
df_CAT.replace({"N.C." : np.nan}, inplace = True)

C:\Users\romai\anaconda3\lib\site-packages\pandas\core\frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [629]:
COL2 = ['Ref_IABTP', 'Distributeur' , 'Ref_Distr', "Famille"]

In [630]:
aa = df_CAT.merge(MAT_LAVIGNA[COL2], #[MAT_LAVIGNA['Ref_Distr'].notnull()]
              left_on = ['Réf. IA BTP', 'Fam', 'Fournisseur', 'Réf. Fournisseur' ],
              right_on = ['Ref_IABTP', 'Famille', 'Distributeur',  'Ref_Distr'],
             indicator=True,
                           how = "left"
            )
aa.head(2)

,Nom du Client,Réf. IA BTP,Fournisseur,Réf. Fournisseur,Fabricant,Réf. Fabricant,Famille_x,Sous-Famille,Désignation,PU_HT,...,Dérogation,Date de Validité Dérogation,Pdf Link,Spécificités techniques,Fam,Ref_IABTP,Distributeur,Ref_Distr,Famille_y,_merge
0,Lavigna,AC1015,Andrety,50355,NaN,NaN,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d18,3.28,...,NaN,NaN,NaN,NaN,Acier Carbone,AC1015,Andrety,50355,Acier Carbone,both
1,Lavigna,AC1016,Andrety,50356,NaN,NaN,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d22,3.53,...,NaN,NaN,NaN,NaN,Acier Carbone,AC1016,Andrety,50356,Acier Carbone,both


In [631]:
df_CAT.shape, aa.shape

((6765, 21), (6765, 26))

In [661]:
C = aa[aa["_merge"] == 'both']
C.shape

(1022, 26)

In [663]:
776 - 1552

-776

In [633]:
p1 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. Fournisseur'].count()]).T

p1.columns = ["Ref_Fo (E-CAT)"]

p1

Ref_Fo (E-CAT)
Fam                   Fournisseur                  
Acier Carbone         Andrety                   110
                      Aredis                      0
                      Cgr                         0
                      First                       0
                      Sgf                         0
...                                             ...
Robinetterie Batiment Sgf                         0
Supportage            Aredis                      0
                      First                       0
                      Mps-Prolians                0
                      Outillage-Mer               0

[64 rows x 1 columns]

In [634]:
p0 = pd.DataFrame([MAT_LAVIGNA.groupby(['Famille', 'Distributeur'], dropna=False)['Ref_Distr'].count()]).T

p0.columns = ["Ref_Fo (MATRICE LAVIGNA)"]

p0

Ref_Fo (MATRICE LAVIGNA)
Famille               Distributeur                           
Acier Carbone         Andrety                             117
                      Aredis                              222
                      Cgr                                 191
                      First                               221
                      Sgf                                 191
...                                                       ...
Robinetterie Batiment Sgf                                 116
Supportage            Aredis                               44
                      First                                45
                      Mps-Prolians                         62
                      Outillage-Mer                        20

[64 rows x 1 columns]

In [635]:
p2 = pd.DataFrame([C.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. Fournisseur'].count()]).T

p2.columns = ["Nbre de Ref_Fo Matchées"]

p2

Nbre de Ref_Fo Matchées
Fam                    Fournisseur                         
Acier Carbone          Andrety                          110
Multicouche            Andrety                          106
                       Richardson                        56
Per Raccord Clarinette Andrety                           48
                       Richardson                        25
Pvc Eu                 Andrety                          127
                       Richardson                       100
Raccord Cuivre         Andrety                          127
                       Richardson                        48
Raccord Laiton         Andrety                           95
                       Richardson                        85
Robinetterie Batiment  Andrety                           47
                       Richardson                        48

In [636]:
D = aa[aa['Réf. IA BTP'] == aa["Ref_IABTP"]]

p3 = pd.DataFrame([D.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. IA BTP'].count()]).T

p3.columns = ["Nbre de Ref. IABTP Matchées"]

p3

Nbre de Ref. IABTP Matchées
Fam                    Fournisseur                             
Acier Carbone          Andrety                              110
Multicouche            Andrety                              106
                       Richardson                            56
Per Raccord Clarinette Andrety                               48
                       Richardson                            25
Pvc Eu                 Andrety                              127
                       Richardson                           100
Raccord Cuivre         Andrety                              127
                       Richardson                            48
Raccord Laiton         Andrety                               95
                       Richardson                            85
Robinetterie Batiment  Andrety                               47
                       Richardson                            48

In [637]:
mask = ['Réf. IA BTP', 'Fam', 'Fournisseur', 'Réf. Fournisseur', 'Ref_IABTP', 'Distributeur', 'Ref_Distr', "Famille_y" ]
aa[mask]

,Réf. IA BTP,Fam,Fournisseur,Réf. Fournisseur,Ref_IABTP,Distributeur,Ref_Distr,Famille_y
0,AC1015,Acier Carbone,Andrety,50355,AC1015,Andrety,50355,Acier Carbone
1,AC1016,Acier Carbone,Andrety,50356,AC1016,Andrety,50356,Acier Carbone
2,AC1245,Acier Carbone,Andrety,47778,AC1245,Andrety,47778,Acier Carbone
3,AC1246,Acier Carbone,Andrety,47780,AC1246,Andrety,47780,Acier Carbone
4,AC1031,Acier Carbone,Andrety,49482,AC1031,Andrety,49482,Acier Carbone
...,...,...,...,...,...,...,...,...
6760,RB1119,Robinetterie Batiment,Sgf,NaN,NaN,NaN,NaN,NaN
6761,RB1121,Robinetterie Batiment,Sgf,NaN,NaN,NaN,NaN,NaN
6762,RB1122,Robinetterie Batiment,Sgf,NaN,NaN,NaN,NaN,NaN
6763,RB1123,Robinetterie Batiment,Sgf,NaN,NaN,NaN,NaN,NaN


In [638]:
I = pd.concat([p1, p0, p3, p2], axis = 1).reset_index()
I.fillna(0, inplace=True)
#I['Réf. Fournisseur'] = I['Réf. Fournisseur'].astype(int)

mask  = ["Ref_Fo (E-CAT)" , "Ref_Fo (MATRICE LAVIGNA)", "Nbre de Ref. IABTP Matchées", "Nbre de Ref_Fo Matchées" ]

I[mask] = I[mask].astype(int)

I.rename(columns = {"level_0" : "Famille",
                   "level_1" : "Fournisseur"}, inplace = True)

I

,Famille,Fournisseur,Ref_Fo (E-CAT),Ref_Fo (MATRICE LAVIGNA),Nbre de Ref. IABTP Matchées,Nbre de Ref_Fo Matchées
0,Acier Carbone,Andrety,110,117,110,110
1,Acier Carbone,Aredis,0,222,0,0
2,Acier Carbone,Cgr,0,191,0,0
3,Acier Carbone,First,0,221,0,0
4,Acier Carbone,Sgf,0,191,0,0
...,...,...,...,...,...,...
59,Robinetterie Batiment,Sgf,0,116,0,0
60,Supportage,Aredis,0,44,0,0
61,Supportage,First,0,45,0,0
62,Supportage,Mps-Prolians,0,62,0,0


In [639]:
q1 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. IA BTP'].count()]).T

q1.columns = ["Ref_IABTP (E-CAT)"]

q1

Ref_IABTP (E-CAT)
Fam                   Fournisseur                     
Acier Carbone         Andrety                      110
                      Aredis                       218
                      Cgr                          171
                      First                        221
                      Sgf                          177
...                                                ...
Robinetterie Batiment Sgf                          114
Supportage            Aredis                        39
                      First                         44
                      Mps-Prolians                  57
                      Outillage-Mer                  9

[64 rows x 1 columns]

In [640]:
q0 = pd.DataFrame([MAT_LAVIGNA.groupby(['Famille', 'Distributeur'], dropna=False)['Ref_IABTP'].count()]).T

q0.columns = ["Ref_IABTP (MATRICE LAVIGNA)"]

q0

Ref_IABTP (MATRICE LAVIGNA)
Famille               Distributeur                              
Acier Carbone         Andrety                                222
                      Aredis                                 222
                      Cgr                                    222
                      First                                  222
                      Sgf                                    222
...                                                          ...
Robinetterie Batiment Sgf                                    186
Supportage            Aredis                                 112
                      First                                  112
                      Mps-Prolians                           112
                      Outillage-Mer                          112

[64 rows x 1 columns]

In [641]:
R = aa[aa["_merge"] == 'left_only']
R.shape

(5743, 26)

In [642]:
w5 = pd.DataFrame([R.groupby(['Fam', 'Fournisseur'])['Réf. IA BTP'].count()]).T

w5.columns = ["Nbre de Ref_IABTP Non Matchées"]

In [643]:
J = pd.concat([q1, q0, p1, p0, p3, p2], axis = 1).reset_index()
J.fillna(0, inplace=True)
#I['Réf. Fournisseur'] = I['Réf. Fournisseur'].astype(int)

mask  = ["Ref_Fo (E-CAT)" , 
         "Ref_IABTP (MATRICE LAVIGNA)",
         "Ref_IABTP (E-CAT)",
         "Ref_Fo (MATRICE LAVIGNA)", "Nbre de Ref. IABTP Matchées", "Nbre de Ref_Fo Matchées" ]

J[mask] = J[mask].astype(int)

J.rename(columns = {"level_0" : "Famille",
                   "level_1" : "Fournisseur"}, inplace = True)

J

,Famille,Fournisseur,Ref_IABTP (E-CAT),Ref_IABTP (MATRICE LAVIGNA),Ref_Fo (E-CAT),Ref_Fo (MATRICE LAVIGNA),Nbre de Ref. IABTP Matchées,Nbre de Ref_Fo Matchées
0,Acier Carbone,Andrety,110,222,110,117,110,110
1,Acier Carbone,Aredis,218,222,0,222,0,0
2,Acier Carbone,Cgr,171,222,0,191,0,0
3,Acier Carbone,First,221,222,0,221,0,0
4,Acier Carbone,Sgf,177,222,0,191,0,0
...,...,...,...,...,...,...,...,...
59,Robinetterie Batiment,Sgf,114,186,0,116,0,0
60,Supportage,Aredis,39,112,0,44,0,0
61,Supportage,First,44,112,0,45,0,0
62,Supportage,Mps-Prolians,57,112,0,62,0,0


In [644]:
K = pd.concat([q1, q0, p1, p0, p3, p2, w5], axis = 1).reset_index()
K.fillna(0, inplace=True)
#I['Réf. Fournisseur'] = I['Réf. Fournisseur'].astype(int)


mask  = ["Ref_Fo (E-CAT)" , 
         "Ref_IABTP (MATRICE LAVIGNA)",
         "Ref_IABTP (E-CAT)",
         "Ref_Fo (MATRICE LAVIGNA)", "Nbre de Ref. IABTP Matchées",
         "Nbre de Ref_Fo Matchées",
        "Nbre de Ref_IABTP Non Matchées",]

K[mask] = K[mask].astype(int)

K.rename(columns = {"level_0" : "Famille",
                   "level_1" : "Fournisseur"}, inplace = True)

K

,Famille,Fournisseur,Ref_IABTP (E-CAT),Ref_IABTP (MATRICE LAVIGNA),Ref_Fo (E-CAT),Ref_Fo (MATRICE LAVIGNA),Nbre de Ref. IABTP Matchées,Nbre de Ref_Fo Matchées,Nbre de Ref_IABTP Non Matchées
0,Acier Carbone,Andrety,110,222,110,117,110,110,0
1,Acier Carbone,Aredis,218,222,0,222,0,0,218
2,Acier Carbone,Cgr,171,222,0,191,0,0,171
3,Acier Carbone,First,221,222,0,221,0,0,221
4,Acier Carbone,Sgf,177,222,0,191,0,0,177
...,...,...,...,...,...,...,...,...,...
59,Robinetterie Batiment,Sgf,114,186,0,116,0,0,114
60,Supportage,Aredis,39,112,0,44,0,0,39
61,Supportage,First,44,112,0,45,0,0,44
62,Supportage,Mps-Prolians,57,112,0,62,0,0,57


In [645]:
filepath = f"{path_controle}/Etat_Des_Lieux_Croisement-{client}__3.xlsx"
    
with pd.ExcelWriter(filepath) as writer:
    
    
    J.to_excel(writer, 
               sheet_name="Etat_lieux_1", 
           index=False) 
    
    K.to_excel(writer, 
               sheet_name="Etat_lieux_2", 
           index=False) 
    
    
    

In [646]:
df_CAT.shape, MAT_LAVIGNA.shape, aa.shape

((6765, 21), (11697, 14), (6765, 26))

In [647]:
aa["_merge"]

0            both
1            both
2            both
3            both
4            both
          ...    
6760    left_only
6761    left_only
6762    left_only
6763    left_only
6764    left_only
Name: _merge, Length: 6765, dtype: category
Categories (3, object): ['left_only', 'right_only', 'both']

In [648]:
# save MATRICE
aa.to_excel(fr"{path_controle}\croisement_MAT_CAT_{client}__3.xlsx", 
               sheet_name="MATRICE", 
           index=False) 

In [649]:
cc = df_CAT.merge(MAT_LAVIGNA[COL2],
              left_on = ['Réf. IA BTP', 'Fam', 'Fournisseur' ],
              right_on = ['Ref_IABTP', 'Famille', 'Distributeur'],
             indicator=True,
                           how = "left"
            )
cc.head(2)

,Nom du Client,Réf. IA BTP,Fournisseur,Réf. Fournisseur,Fabricant,Réf. Fabricant,Famille_x,Sous-Famille,Désignation,PU_HT,...,Dérogation,Date de Validité Dérogation,Pdf Link,Spécificités techniques,Fam,Ref_IABTP,Distributeur,Ref_Distr,Famille_y,_merge
0,Lavigna,AC1015,Andrety,50355,NaN,NaN,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d18,3.28,...,NaN,NaN,NaN,NaN,Acier Carbone,AC1015,Andrety,50355,Acier Carbone,both
1,Lavigna,AC1016,Andrety,50356,NaN,NaN,Plomberie,Acier Carbone;Bouchon à sertir femelle,Bouchon à sertir femelle - d22,3.53,...,NaN,NaN,NaN,NaN,Acier Carbone,AC1016,Andrety,50356,Acier Carbone,both


In [650]:
df_CAT.shape, cc.shape, aa.shape


((6765, 21), (7153, 26), (6765, 26))

In [651]:
D = cc[cc["_merge"] == 'both']
D.shape

(7153, 26)

In [652]:
p0 = pd.DataFrame([MAT_LAVIGNA.groupby(['Famille', 'Distributeur'], dropna=False)['Ref_Distr'].count()]).T

p0.columns = ["Ref_Fo (MATRICE LAVIGNA)"]

p1 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. Fournisseur'].count()]).T

p1.columns = ["Ref_Fo (E-CAT)"]


q0 = pd.DataFrame([MAT_LAVIGNA.groupby(['Famille', 'Distributeur'], dropna=False)['Ref_IABTP'].count()]).T

q0.columns = ["Ref_IABTP (MATRICE LAVIGNA)"]


q1 = pd.DataFrame([df_CAT.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. IA BTP'].count()]).T

q1.columns = ["Ref_IABTP (E-CAT)"]

In [672]:
7153 - 6765, 8639 - 7836, 1552 - 776, 803 - 776, 8639 - 7863, 5743 + 1022, 7153-388

(388, 803, 776, 27, 776, 6765, 6765)

In [653]:
p2 = pd.DataFrame([D.groupby(['Fam', 'Fournisseur'], dropna=False)['Réf. IA BTP'].count()]).T

p2.columns = ["Nbre de Ref_IABTP Matchées"]

p2

Nbre de Ref_IABTP Matchées
Fam                   Fournisseur                              
Acier Carbone         Andrety                               110
                      Aredis                                218
                      Cgr                                   171
                      First                                 221
                      Sgf                                   177
...                                                         ...
Robinetterie Batiment Sgf                                   114
Supportage            Aredis                                 39
                      First                                  44
                      Mps-Prolians                           57
                      Outillage-Mer                           9

[64 rows x 1 columns]

In [654]:
J = pd.concat([q1, q0, p1, p0, p2], axis = 1).reset_index()
J.fillna(0, inplace=True)
#I['Réf. Fournisseur'] = I['Réf. Fournisseur'].astype(int)


mask  = ["Ref_Fo (E-CAT)" , 
         "Ref_IABTP (MATRICE LAVIGNA)",
         "Ref_IABTP (E-CAT)",
         "Ref_Fo (MATRICE LAVIGNA)", "Nbre de Ref_IABTP Matchées" ]

J[mask] = J[mask].astype(int)


J.rename(columns = {"level_0" : "Famille",
                   "level_1" : "Fournisseur"}, inplace = True)


J

,Famille,Fournisseur,Ref_IABTP (E-CAT),Ref_IABTP (MATRICE LAVIGNA),Ref_Fo (E-CAT),Ref_Fo (MATRICE LAVIGNA),Nbre de Ref_IABTP Matchées
0,Acier Carbone,Andrety,110,222,110,117,110
1,Acier Carbone,Aredis,218,222,0,222,218
2,Acier Carbone,Cgr,171,222,0,191,171
3,Acier Carbone,First,221,222,0,221,221
4,Acier Carbone,Sgf,177,222,0,191,177
...,...,...,...,...,...,...,...
59,Robinetterie Batiment,Sgf,114,186,0,116,114
60,Supportage,Aredis,39,112,0,44,39
61,Supportage,First,44,112,0,45,44
62,Supportage,Mps-Prolians,57,112,0,62,57


In [655]:
E = cc[cc["_merge"] == 'left_only']
E.shape

(0, 26)

In [656]:
p5 = pd.DataFrame([E.groupby(['Fam', 'Fournisseur'])['Réf. IA BTP'].count()]).T

p5.columns = ["Nbre de Ref_IABTP Non Matchées"]

p5

,,Nbre de Ref_IABTP Non Matchées
Fam,Fournisseur,


In [657]:
K = pd.concat([q1, q0, p1, p0, p2, p5], axis = 1).reset_index()
K.fillna(0, inplace=True)
#I['Réf. Fournisseur'] = I['Réf. Fournisseur'].astype(int)


mask  = ["Ref_Fo (E-CAT)" , 
         "Ref_IABTP (MATRICE LAVIGNA)",
         "Ref_IABTP (E-CAT)",
         "Ref_Fo (MATRICE LAVIGNA)", "Nbre de Ref_IABTP Matchées" ,
         "Nbre de Ref_IABTP Non Matchées"]

K[mask] = K[mask].astype(int)


K.rename(columns = {"level_0" : "Famille",
                   "level_1" : "Fournisseur"}, inplace = True)



K

,Famille,Fournisseur,Ref_IABTP (E-CAT),Ref_IABTP (MATRICE LAVIGNA),Ref_Fo (E-CAT),Ref_Fo (MATRICE LAVIGNA),Nbre de Ref_IABTP Matchées,Nbre de Ref_IABTP Non Matchées
0,Acier Carbone,Andrety,110,222,110,117,110,0
1,Acier Carbone,Aredis,218,222,0,222,218,0
2,Acier Carbone,Cgr,171,222,0,191,171,0
3,Acier Carbone,First,221,222,0,221,221,0
4,Acier Carbone,Sgf,177,222,0,191,177,0
...,...,...,...,...,...,...,...,...
59,Robinetterie Batiment,Sgf,114,186,0,116,114,0
60,Supportage,Aredis,39,112,0,44,39,0
61,Supportage,First,44,112,0,45,44,0
62,Supportage,Mps-Prolians,57,112,0,62,57,0


In [658]:
filepath = f"{path_controle}/Etat_Des_Lieux_Croisement-{client}__Ref_IABTP__3.xlsx"
    
with pd.ExcelWriter(filepath) as writer:
     
    
    J.to_excel(writer, 
               sheet_name="Etat_lieux", 
           index=False) 
    
    
    K.to_excel(writer, 
               sheet_name="Etat_lieux_1", 
           index=False) 
    
    
    

In [660]:
D.shape, E.shape

((7153, 26), (0, 26))

In [141]:
def merge_data_on_col(df1 = pd.DataFrame(), df2= pd.DataFrame(), col = ["Ref_IABTP"],
                      dataname1 = '_LAV', dataname2 = '_MAT'):
    
    
    #print(col, len(col))
    if len(col) > 1 :
        COL1 = col[1]
    elif len(col) ==1 :
        COL1 = col[0]
        #print(COL1)
           
    res = df1[~(df1[COL1].isnull()) ].merge(df2[~(df2[COL1].isnull()) ],
              on = col,
             indicator=True,
                           how = "outer",
              suffixes=(dataname1, dataname2))
    
    
        
    b = res[res["_merge"] == 'both']
    bc = b[COL1].count()
    
    #print(df1[~(df1[COL1].isnull()) ].shape, df2[~(df2[COL1].isnull()) ].shape, b.shape)

    


    l = res[res["_merge"] == 'left_only'] # CAT
    lc = l[COL1].count()

    r = res[res["_merge"] == 'right_only'] # Corresp
    rc = r[COL1].count()

    return res, bc, lc, rc   #, b, l, r

In [142]:
dt_perso = MAT_LAVIGNA[mask_cols]
dt_whole = df_MAT[mask_cols]

res_1, bc_1, lc_1, rc_1 = merge_data_on_col(df1 = dt_perso,
                  df2= dt_whole,
                  col = ['Ref_IABTP', 'Ref_Distr'], #"Ref_Distr",
                  dataname1 = '_LAV', 
                  dataname2 = '_MAT')

In [148]:
BB = res_1[res_1["_merge"] == 'both']
BC = BB['Ref_Distr'].count()

LL = res_1[res_1["_merge"] == 'left_only']
LC = LL['Ref_Distr'].count()

RR = res_1[res_1["_merge"] == 'right_only']
RC = RR['Ref_Distr'].count()


L = []
L_1 = []
L_2 = []

L_1.append([dt_perso["Ref_Distr"].count(), dt_whole["Ref_Distr"].count(), BC, LC, RC])

L_1

[[5212, 16737, 5212, 0, 11525]]

In [43]:
filepath = f"{path_controle}Contrôle 6/Etat_Des_Lieux_MAT-{client}.xlsx"
    
with pd.ExcelWriter(filepath) as writer:
    
    Controle_0.to_excel(writer,
               sheet_name="MATCHING Sur Ref_IABTP", 
           index=False) 
    
    
    Controle_1.to_excel(writer, 
               sheet_name="MATCHING Sur Ref_Distr", 
           index=False) 
    
    
    

In [44]:
S = []
T = []
V = []

#  Ref_IABTP : Check if all Ref_IABTP are commons in MATRICE & MATRICE ES
for i in range(len(L)):
    a = L[i][2]
    b = L[i][3]
    c = L[i][4]
    cond = (a == b) & (b == c)
    S.append(cond)
    
if np.all(S):
    print("La MATRICE ES est bien construite; Aucune donnée n'est perdue")
    
#  Ref_Distr : Check if all Ref_Distr are commons in MATRICE & MATRICE ES

for i in range(len(L_1)):
    a = L_1[i][2]
    b = L_1[i][3]
    c = L_1[i][4]
    cond = (a == b) & (b == c)
    T.append(cond)
    
if np.all(T):
    print("La MATRICE ES est bien construite; Aucune donnée n'est perdue")
    

La MATRICE ES est bien construite; Aucune donnée n'est perdue
La MATRICE ES est bien construite; Aucune donnée n'est perdue


In [45]:
MAT_LAVIGNA.shape

(11736, 13)